In [1]:
import yfinance as yf

data = yf.download(["^NSEI", "^GSPC", "^N225", "^FTSE"], start="2024-01-01")


/var/folders/jm/hq0bsc_j09q7l0wkv9m4k3c00000gn/T/ipykernel_41752/1971868049.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(["^NSEI", "^GSPC", "^N225", "^FTSE"], start="2024-01-01")
[*********************100%***********************]  4 of 4 completed


In [2]:
import pandas as pd

# Load your GKG data (already cleaned)
df = pd.read_csv("gdelt_events_clean.csv")

# Extract country from the 'V2Locations' column
# Example pattern: '1#India#IN#IN##20#77#IN#2836'
def extract_country(loc):
    if isinstance(loc, str) and "#" in loc:
        try:
            return loc.split("#")[1]
        except:
            return None
    return None

df["country"] = df["V2Locations"].apply(extract_country)

# Convert DATE to proper date
df["DATE"] = pd.to_datetime(df["DATE"].astype(str).str[:8], format="%Y%m%d")

# Convert Tone to numeric
df["Tone_Avg"] = df["V2Tone"].str.split(",").str[0].astype(float)

# Group by date and country
daily_tone = df.groupby(["DATE", "country"])["Tone_Avg"].mean().reset_index()
daily_tone.head()


FileNotFoundError: [Errno 2] No such file or directory: 'gdelt_events_clean.csv'

In [ ]:
countries = ["India", "United States", "China", "Japan", "United Kingdom", "Germany"]
daily_tone = daily_tone[daily_tone["country"].isin(countries)]


In [ ]:
import yfinance as yf

# Example: get NIFTY and S&P 500
markets = yf.download(["^NSEI", "^GSPC"], start="2024-01-01")["Adj Close"]
markets = markets.reset_index().rename(columns={"Date": "DATE"})

# Merge sentiment with market returns
merged = pd.merge(daily_tone, markets, on="DATE", how="inner")

# Compute daily returns
merged["NSEI_Return"] = merged["^NSEI"].pct_change()
merged["SP500_Return"] = merged["^GSPC"].pct_change()


In [ ]:
merged.groupby("country")[["Tone_Avg", "SP500_Return"]].corr().unstack().iloc[:,1]


In [ ]:
import statsmodels.api as sm

X = merged["Tone_Avg"]
y = merged["NSEI_Return"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())


In [ ]:
import matplotlib.pyplot as plt

for c in countries:
    subset = merged[merged["country"] == c]
    plt.plot(subset["DATE"], subset["Tone_Avg"], label=f"{c} Sentiment")

plt.legend()
plt.title("Global News Sentiment over Time")
plt.show()
